<a href="https://colab.research.google.com/github/vitorsr/ccd/blob/master/baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Definição do Baseline

## Bibliotecas Utilizadas

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

## Extração de Dados

In [2]:
!wget -q -O ccd_2019.zip https://www.dropbox.com/s/7rriacb7c6vzf3m/ccd_2019.zip

!unzip ccd_2019.zip


Archive:  ccd_2019.zip
  inflating: bdmep_meta.csv          
  inflating: __MACOSX/._bdmep_meta.csv  
  inflating: inmetr.csv              
  inflating: __MACOSX/._inmetr.csv   


In [3]:
!wget "https://onedrive.live.com/download?cid=BCBADB0836C97273&resid=BCBADB0836C97273%21115485&authkey=AJlb41BWz0YiBd8" -O 2019.zip
!unzip 2019.zip

--2019-10-26 16:16:27--  https://onedrive.live.com/download?cid=BCBADB0836C97273&resid=BCBADB0836C97273%21115485&authkey=AJlb41BWz0YiBd8
Resolving onedrive.live.com (onedrive.live.com)... 13.107.42.13
Connecting to onedrive.live.com (onedrive.live.com)|13.107.42.13|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://c9quda.bn.files.1drv.com/y4m2Kvy_qD0HBkQ1_oxH0Et6OaCDeQHUxLK1ofvSz4AkZx76nfOUftoRMsMg4dnMrpqFxzBvxM76t6k6_EEWrdaTJB4av_wHcfoCmA9-utglFRe5XHueCzC6P3FB1RkE9fmO3xgnt5YdH1LIEQxj3410W-_jEWbw5qaWxGCvBkri1Sani1Ck4MeeQ-ROzC5hwsbjPhrotx00xv4SxjtqLuoFw/2019.zip?download&psid=1 [following]
--2019-10-26 16:16:28--  https://c9quda.bn.files.1drv.com/y4m2Kvy_qD0HBkQ1_oxH0Et6OaCDeQHUxLK1ofvSz4AkZx76nfOUftoRMsMg4dnMrpqFxzBvxM76t6k6_EEWrdaTJB4av_wHcfoCmA9-utglFRe5XHueCzC6P3FB1RkE9fmO3xgnt5YdH1LIEQxj3410W-_jEWbw5qaWxGCvBkri1Sani1Ck4MeeQ-ROzC5hwsbjPhrotx00xv4SxjtqLuoFw/2019.zip?download&psid=1
Resolving c9quda.bn.files.1drv.com (c9quda.bn.files.1drv.com)... 1

In [0]:
directory = '2019'
files = []
for filename in os.listdir(directory):
    if filename.endswith(".csv"): 
        files.append(filename)
        continue
    else:
        continue

In [5]:
df_2019 = pd.DataFrame()
for file in files:
    df_aux = pd.read_csv(directory+'/'+file, delimiter=',')
    df_2019 = pd.concat([df_aux, df_2019], sort=True)
id_2019 = pd.unique(df_2019['id'])
id_2019

array([83632, 82588, 82992, 82983, 82676, 82191, 82659, 83557, 83377,
       83049, 83288, 82331, 83927, 83623, 83698, 82098, 82198, 83811,
       83184, 83368, 82024, 82042, 82970, 82336, 83536, 83936, 83442,
       82765, 83738, 82975, 82178, 83286, 82382, 82141, 82689, 83807,
       82493, 82586, 83550, 82723, 83844, 82890, 83639, 83997, 82317,
       83592, 82326, 83190, 83912, 83249, 82188, 82184, 83338, 83376,
       82882, 82893, 82487, 83805, 83064, 82564, 82795, 83916, 82106,
       82590, 82990, 83192, 83096, 82476, 83736, 83781, 82392, 83386,
       83591, 82578, 83522, 83767, 82181, 82533, 83090, 82145, 82212,
       83967, 82780, 83013, 82361, 83334, 83221, 83244, 82792, 83676,
       82753, 83942, 82915, 82994, 82445, 83687, 83361, 82996, 83538,
       82863, 83744, 82798, 82246, 83964, 83229, 83428, 83577, 83526,
       83441, 82376, 83695, 83483, 83919, 83669, 83379, 83579, 82596,
       82693, 83007, 83714, 82610, 82598, 83683, 83851, 82976, 83470,
       82861, 82460,

<br><b>variável | descrição | unidade </b></br>
<br>date | data e hora da coleta             -
<br> id | ID da estação de coleta             -
<br>prec | precipitação |       mm
<br>tair | temperatura do ar  | graus Celsius
<br>tw | temperatura de bulbo úmido |  graus Celsius
<br>tmax | temperatura máxima do ar  | graus Celsius
<br>tmin | temperatura mínima do ar  | graus Celsius
<br>urmax | umidade relativa máxima  |             %
<br>patm | pressão atmosférica         |    hPa
<br>pnmm | pressão atmosférica média ao nível do mar  |           hPa
<br>wd | direção do vento        |   graus
<br>wsmax  |  rajadas de vento    |         m/s
<br>n |  horas de sol          |     h
<br>cc |   cobertura de nuvens    |           -
<br>evap  |   evaporação        |      mm
<br>ur  |   umidade relativa          |     %
<br>ws   |   velocidade do vento       |      m/s

In [6]:
df = pd.read_csv("inmetr.csv")
df.head()

,date,id,prec,tair,tw,tmax,tmin,urmax,patm,pnmm,wd,wsmax,n,cc,evap,ur,ws
0,1970-05-04 00:00:00,83010,NaN,NaN,NaN,32.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,89.75,NaN
1,1970-05-04 12:00:00,83010,NaN,25.3,24.0,NaN,23.7,90.0,1005.9,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN
2,1970-05-04 18:00:00,83010,NaN,29.2,27.1,NaN,NaN,85.0,1004.2,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN
3,1970-05-05 00:00:00,83010,NaN,25.0,27.0,32.6,NaN,92.0,1007.5,NaN,NaN,NaN,NaN,8.0,2.4,88.25,NaN
4,1970-05-05 12:00:00,83010,0.0,25.9,24.8,NaN,23.0,91.0,1006.4,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN


In [7]:
df_m = pd.read_csv("bdmep_meta.csv")
df_m.head()

,id,lon,lat,alt,name,state,uf,time_zone,offset_utc,time_zone.1,offset_utc.1
0,83010,-68.733333,-11.016667,260.00,Brasiléia,Acre,AC,America/Rio_Branco,-5,America/Rio_Branco,-5
1,82704,-72.666667,-7.633333,170.00,Cruzeiro do Sul,Acre,AC,America/Rio_Branco,-5,America/Rio_Branco,-5
2,82915,-67.800000,-9.966667,160.00,Rio Branco,Acre,AC,America/Rio_Branco,-5,America/Rio_Branco,-5
3,82807,-70.766667,-8.166667,190.00,Tarauacá,Acre,AC,America/Rio_Branco,-5,America/Rio_Branco,-5
4,83098,-36.166667,-10.150000,56.13,Coruripe,Alagoas,AL,America/Maceio,-3,America/Maceio,-3


## Processamento de Dados Históricos

Os dados apresentados abragem o período entre os anos de 1970 e 2018.

Filtrando apenas para o Mês de Agosto:

In [8]:
mask = (df['date'].str.split('-',n=2,expand = True)[1]=='08')
df = df.loc[mask]
df.size

20944306

Filtrando apenas as Estações ainda Existentes em 2019:

In [9]:
mask = (df['id'].isin(id_2019))
df = df.loc[mask]
df.size

13499360

Extraindo o Ano da Coluna Date:

In [10]:
df['date'] = df['date'].str.split('-',n=1,expand = True)[1]
df.head()

,date,id,prec,tair,tw,tmax,tmin,urmax,patm,pnmm,wd,wsmax,n,cc,evap,ur,ws
53402,08-01 00:00:00,82915,NaN,25.0,22.8,35.0,NaN,84.0,1010.6,NaN,NaN,NaN,5.6,4.0,3.2,74.75,0.0
53403,08-01 12:00:00,82915,0.0,22.2,21.6,NaN,19.7,94.0,1000.6,NaN,0.0,0.0,NaN,7.0,NaN,NaN,NaN
53404,08-01 18:00:00,82915,NaN,33.8,27.0,NaN,NaN,57.0,999.2,NaN,0.0,0.0,NaN,7.0,NaN,NaN,NaN
53405,08-02 00:00:00,82915,NaN,26.8,24.8,33.8,NaN,74.0,996.6,NaN,NaN,NaN,3.5,6.0,2.1,83.50,NaN
53406,08-02 12:00:00,82915,21.9,11.8,11.0,NaN,11.2,90.0,1009.0,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN


Extraindo a Hora de Medição da Coluna Date:

In [11]:
df['date'] = df['date'].str.split(' ',n=1,expand = True)[0]
df.head()

,date,id,prec,tair,tw,tmax,tmin,urmax,patm,pnmm,wd,wsmax,n,cc,evap,ur,ws
53402,08-01,82915,NaN,25.0,22.8,35.0,NaN,84.0,1010.6,NaN,NaN,NaN,5.6,4.0,3.2,74.75,0.0
53403,08-01,82915,0.0,22.2,21.6,NaN,19.7,94.0,1000.6,NaN,0.0,0.0,NaN,7.0,NaN,NaN,NaN
53404,08-01,82915,NaN,33.8,27.0,NaN,NaN,57.0,999.2,NaN,0.0,0.0,NaN,7.0,NaN,NaN,NaN
53405,08-02,82915,NaN,26.8,24.8,33.8,NaN,74.0,996.6,NaN,NaN,NaN,3.5,6.0,2.1,83.50,NaN
53406,08-02,82915,21.9,11.8,11.0,NaN,11.2,90.0,1009.0,NaN,NaN,NaN,NaN,8.0,NaN,NaN,NaN


Group By por Data e Estação:

In [12]:
df = df.groupby(by=['date','id']).mean()
df.head(1000)

prec       tair         tw  ...      evap         ur        ws
date  id                                     ...                               
08-01 82024  6.219512  27.235246  24.882883  ...  2.597143  84.051471  1.025690
      82042  4.133333  26.858163  24.436145  ...  2.345455  83.840000  0.467799
      82098  3.716327  28.204027  25.416779  ...  2.765306  81.761364  1.290667
      82106  4.888889  25.806329  24.044521  ...  1.298000  88.738372  1.182592
      82113  6.386538  26.638000  24.857746  ...  1.847368  87.191860  1.266859
...               ...        ...        ...  ...       ...        ...       ...
08-06 82336  3.765116  28.311024  25.521930  ...  3.527778  80.323529  1.007813
      82353  1.214815  28.338961  24.939610  ...  2.410870  77.088889  1.326016
      82361  1.113953  28.308462  25.276378  ...  3.400000  78.694444  1.404924
      82376  0.081395  28.585217  24.247826  ...  4.674419  72.789474  2.037037
      82382  0.159524  28.420909  23.314545  ...  5.735714  68.409722  2.142157

[1000 rows x 15 columns]

## Processamento de Dados Atuais

In [13]:
df_2019.head()

,cc,date,evap,id,n,patm,pnmm,prec,request_status,tair,tmax,tmin,tw,ur,urmax,wd,ws,wsmax
0,-9999.0,2019-01-01T00:00:00Z,3.0,83632,-9999.0,-9999.0,-9999.0,-9999.0,Success: (200) OK,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999,-9999,-9999.0,-9999.0
1,-9999.0,2019-01-01T12:00:00Z,-9999.0,83632,-9999.0,-9999.0,-9999.0,6.2,Success: (200) OK,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999,-9999,-9999.0,-9999.0
2,-9999.0,2019-01-02T00:00:00Z,2.0,83632,-9999.0,-9999.0,-9999.0,-9999.0,Success: (200) OK,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999,-9999,-9999.0,-9999.0
3,-9999.0,2019-01-02T12:00:00Z,-9999.0,83632,-9999.0,-9999.0,-9999.0,0.0,Success: (200) OK,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999,-9999,-9999.0,-9999.0
4,-9999.0,2019-01-03T00:00:00Z,3.5,83632,-9999.0,-9999.0,-9999.0,-9999.0,Success: (200) OK,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999,-9999,-9999.0,-9999.0


In [14]:
df_2019 = df_2019.drop(['request_status'],axis=1)
df_2019.head()

,cc,date,evap,id,n,patm,pnmm,prec,tair,tmax,tmin,tw,ur,urmax,wd,ws,wsmax
0,-9999.0,2019-01-01T00:00:00Z,3.0,83632,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999,-9999,-9999.0,-9999.0
1,-9999.0,2019-01-01T12:00:00Z,-9999.0,83632,-9999.0,-9999.0,-9999.0,6.2,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999,-9999,-9999.0,-9999.0
2,-9999.0,2019-01-02T00:00:00Z,2.0,83632,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999,-9999,-9999.0,-9999.0
3,-9999.0,2019-01-02T12:00:00Z,-9999.0,83632,-9999.0,-9999.0,-9999.0,0.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999,-9999,-9999.0,-9999.0
4,-9999.0,2019-01-03T00:00:00Z,3.5,83632,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,-9999,-9999,-9999.0,-9999.0


Substituindo Valores Inválidos (-9999.0):

In [15]:
df_2019 = df_2019.replace(-9999.0,np.nan)
df_2019.head()

,cc,date,evap,id,n,patm,pnmm,prec,tair,tmax,tmin,tw,ur,urmax,wd,ws,wsmax
0,NaN,2019-01-01T00:00:00Z,3.0,83632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2019-01-01T12:00:00Z,NaN,83632,NaN,NaN,NaN,6.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,2019-01-02T00:00:00Z,2.0,83632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,2019-01-02T12:00:00Z,NaN,83632,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,2019-01-03T00:00:00Z,3.5,83632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Segue-se agora o mesmo Processo feito para os Dados Históricos.

Filtrando apenas para o Mês de Agosto:

In [16]:
mask = (df_2019['date'].str.split('-',n=2,expand = True)[1]=='08')
df_2019 = df_2019.loc[mask]
df_2019.size

250733

Os Dados aqui já contém naturalmente apenas Estações Existentes em 2019.

O Ano também já é naturalmente o de 2019, mas seu valor será extraído para fins de padronização.

In [17]:
df_2019['date'] = df_2019['date'].str.split('-',n=1,expand = True)[1]
df_2019.head()

,cc,date,evap,id,n,patm,pnmm,prec,tair,tmax,tmin,tw,ur,urmax,wd,ws,wsmax
302,NaN,08-01T00:00:00Z,2.1,83632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
303,NaN,08-01T12:00:00Z,NaN,83632,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304,NaN,08-02T00:00:00Z,1.8,83632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305,NaN,08-02T12:00:00Z,NaN,83632,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306,NaN,08-03T00:00:00Z,2.3,83632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Extraindo a Hora de Medição da Coluna Date:

In [18]:
df_2019['date'] = df_2019['date'].str.split('T',n=1,expand = True)[0]
df_2019.head()

,cc,date,evap,id,n,patm,pnmm,prec,tair,tmax,tmin,tw,ur,urmax,wd,ws,wsmax
302,NaN,08-01,2.1,83632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
303,NaN,08-01,NaN,83632,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
304,NaN,08-02,1.8,83632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
305,NaN,08-02,NaN,83632,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306,NaN,08-03,2.3,83632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Group By por Data e Estação:

In [19]:
df_2019 = df_2019.groupby(by=['date','id']).mean()
df_2019.head(1000)

cc  evap     n  ...         wd        ws     wsmax
date  id                            ...                               
08-01 82024   6.250000   NaN   9.6  ...  11.500000  0.514440  0.514440
      82098   6.333333   3.7   8.9  ...   7.666667  1.666667  2.000000
      82106   9.000000   NaN   NaN  ...   4.500000  0.514440  0.514440
      82141   5.000000   0.3  10.5  ...  14.333333  2.733333  2.733333
      82145   4.333333   1.0   9.8  ...  15.000000  0.533333  0.533333
...                ...   ...   ...  ...        ...       ...       ...
08-06 83565   9.666667   2.2   2.5  ...  11.000000  1.333333  2.000000
      83579   6.333333   4.2   0.5  ...        NaN       NaN       NaN
      83587   9.000000   4.7   1.3  ...   7.666667  2.600000  2.700000
      83589  10.000000   3.9   NaN  ...        NaN       NaN       NaN
      83592   8.333333   3.4   0.1  ...  14.000000  1.666667  1.333333

[1000 rows x 15 columns]

## Cálculo do Baseline

In [23]:
df.head()

prec       tair         tw  ...      evap         ur        ws
date  id                                     ...                               
08-01 82024  6.219512  27.235246  24.882883  ...  2.597143  84.051471  1.025690
      82042  4.133333  26.858163  24.436145  ...  2.345455  83.840000  0.467799
      82098  3.716327  28.204027  25.416779  ...  2.765306  81.761364  1.290667
      82106  4.888889  25.806329  24.044521  ...  1.298000  88.738372  1.182592
      82113  6.386538  26.638000  24.857746  ...  1.847368  87.191860  1.266859

[5 rows x 15 columns]

In [24]:
df_2019.head()

cc  evap     n  ...         wd        ws     wsmax
date  id                           ...                               
08-01 82024  6.250000   NaN   9.6  ...  11.500000  0.514440  0.514440
      82098  6.333333   3.7   8.9  ...   7.666667  1.666667  2.000000
      82106  9.000000   NaN   NaN  ...   4.500000  0.514440  0.514440
      82141  5.000000   0.3  10.5  ...  14.333333  2.733333  2.733333
      82145  4.333333   1.0   9.8  ...  15.000000  0.533333  0.533333

[5 rows x 15 columns]

In [28]:
pd.merge(df_2019,df,on=['date','id'])

cc_x  evap_x   n_x  ...    evap_y       ur_y      ws_y
date  id                              ...                               
08-01 82024   6.250000     NaN   9.6  ...  2.597143  84.051471  1.025690
      82098   6.333333     3.7   8.9  ...  2.765306  81.761364  1.290667
      82106   9.000000     NaN   NaN  ...  1.298000  88.738372  1.182592
      82141   5.000000     0.3  10.5  ...  2.694444  79.305000  2.241212
      82145   4.333333     1.0   9.8  ...  2.139535  85.653846  1.192929
...                ...     ...   ...  ...       ...        ...       ...
08-31 83964   9.333333     1.6   0.9  ...  2.211628  76.226744  2.173956
      83967  10.000000     2.0   0.0  ...  2.366667  78.919811  2.122404
      83980   7.000000     0.1   6.9  ...  3.320408  74.037234  3.238373
      83985   9.333333     0.5   4.3  ...  2.405556  83.562500  3.094286
      83997   6.666667     1.0   NaN  ...  2.275556  84.316667  4.398374

[5409 rows x 30 columns]